# Xopt Evaluator Basic Usage 

The `Evaluator` handles the execution of the user-provided `function` with optional `function_kwags`, asyncrhonously and parallel, with exception handling. 

In [1]:
# needed for macos
import platform
if platform.system() == "Darwin": import multiprocessing;multiprocessing.set_start_method("fork")


In [2]:
from xopt import Xopt, Evaluator, VOCS
from xopt.generators.random import RandomGenerator

import pandas as pd

from time import sleep
from numpy.random import randint

from typing import Dict

import numpy as np
np.random.seed(666) # for reproducibility

Define a custom function `f(inputs: Dict) -> outputs: Dict`. 

In [3]:
def f(inputs: Dict, enable_errors=True) -> Dict:

    sleep(randint(1, 5)*.1)  # simulate computation time
    # Make some occasional errors
    if enable_errors and np.any(inputs["x"] > 0.8):
        raise ValueError("x > 0.8")

    return {"f1": inputs["x"] ** 2 + inputs["y"] ** 2}

Define variables, objectives, constraints, and other settings (VOCS)

In [4]:
vocs = VOCS(variables={"x": [0, 1], "y": [0, 1]}, objectives={"f1": "MINIMIZE"})
vocs



VOCS(variables={'x': [0.0, 1.0], 'y': [0.0, 1.0]}, constraints={}, objectives={'f1': 'MINIMIZE'}, constants={}, observables=[])

This can be used to make some random inputs for evaluating the function. 

In [5]:
in1 = vocs.random_inputs()

f(in1, enable_errors=False)

{'f1': 0.11401572022703582}

In [6]:
# Add in occasional errors. 
try:
    f({"x": 1, "y": 0})
except Exception as ex:
    print(f"Caught error in f: {ex}")

Caught error in f: x > 0.8


In [7]:
# Create Evaluator
ev = Evaluator(function=f)

In [8]:
# Single input evaluation
ev.evaluate(in1)

{'f1': 0.11401572022703582,
 'xopt_runtime': 0.20760420000000046,
 'xopt_error': False}

In [9]:
# Dataframe evaluation
in10 = pd.DataFrame({
    "x":np.linspace(0,1,10),
    "y":np.linspace(0,1,10)
})
ev.evaluate_data(in10)


,f1,xopt_runtime,xopt_error,xopt_error_str
0,0.000000,0.310463,False,NaN
1,0.024691,0.104879,False,NaN
2,0.098765,0.210034,False,NaN
3,0.222222,0.404884,False,NaN
4,0.395062,0.315116,False,NaN
5,0.617284,0.404903,False,NaN
6,0.888889,0.105001,False,NaN
7,1.209877,0.404980,False,NaN
8,NaN,0.105407,True,"Traceback (most recent call last):\n File ""C:..."
9,NaN,0.314808,True,"Traceback (most recent call last):\n File ""C:..."


In [10]:
# Dataframe submission (returns futures dict)
futures = ev.submit_data(in10)
for future in futures:
    print(future.result())

{'f1': 0.0, 'xopt_runtime': 0.2062811999999994, 'xopt_error': False}
{'f1': 0.024691358024691357, 'xopt_runtime': 0.40597700000000003, 'xopt_error': False}
{'f1': 0.09876543209876543, 'xopt_runtime': 0.1049781000000003, 'xopt_error': False}
{'f1': 0.2222222222222222, 'xopt_runtime': 0.31498069999999956, 'xopt_error': False}
{'f1': 0.3950617283950617, 'xopt_runtime': 0.10490089999999874, 'xopt_error': False}
{'f1': 0.617283950617284, 'xopt_runtime': 0.40494879999999966, 'xopt_error': False}
{'f1': 0.8888888888888888, 'xopt_runtime': 0.20991550000000103, 'xopt_error': False}
{'f1': 1.2098765432098761, 'xopt_runtime': 0.31492969999999865, 'xopt_error': False}
{'xopt_runtime': 0.10523910000000036, 'xopt_error': True, 'xopt_error_str': 'Traceback (most recent call last):\n  File "C:\\Users\\Ryan Roussel\\Documents\\GitHub\\Xopt\\xopt\\utils.py", line 138, in safe_call\n    result = func(*args, **kwargs)\n  File "C:\\Users\\Ryan Roussel\\AppData\\Local\\Temp\\ipykernel_30412\\1163003050.py",

In [11]:
# Dataframe evaluation, vectorized
ev.vectorized = True
ev.evaluate_data(in10)


,xopt_runtime,xopt_error,xopt_error_str
0,0.207302,True,"Traceback (most recent call last):\n File ""C:..."
1,0.207302,True,"Traceback (most recent call last):\n File ""C:..."
2,0.207302,True,"Traceback (most recent call last):\n File ""C:..."
3,0.207302,True,"Traceback (most recent call last):\n File ""C:..."
4,0.207302,True,"Traceback (most recent call last):\n File ""C:..."
5,0.207302,True,"Traceback (most recent call last):\n File ""C:..."
6,0.207302,True,"Traceback (most recent call last):\n File ""C:..."
7,0.207302,True,"Traceback (most recent call last):\n File ""C:..."
8,0.207302,True,"Traceback (most recent call last):\n File ""C:..."
9,0.207302,True,"Traceback (most recent call last):\n File ""C:..."


In [12]:
# Vectorized submission. This returns a single future.
ev.vectorized = True
futures = ev.submit_data(in10)
len(futures)

1

In [13]:
futures[0].result()

{'xopt_runtime': 0.20739859999999943,
 'xopt_error': True,
 'xopt_error_str': 'Traceback (most recent call last):\n  File "C:\\Users\\Ryan Roussel\\Documents\\GitHub\\Xopt\\xopt\\utils.py", line 138, in safe_call\n    result = func(*args, **kwargs)\n  File "C:\\Users\\Ryan Roussel\\AppData\\Local\\Temp\\ipykernel_30412\\1163003050.py", line 6, in f\n    raise ValueError("x > 0.8")\nValueError: x > 0.8\n'}

In [14]:
# Collect in a dataframe
res = futures[0].result()
# If there is an error, all outputs are spoiled.
if res['xopt_error']:
    res = [res]
pd.DataFrame(res)

,xopt_runtime,xopt_error,xopt_error_str
0,0.207399,True,"Traceback (most recent call last):\n File ""C:..."


# Executors

In [15]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
MAX_WORKERS = 10

In [16]:
# Create Executor insance
executor = ProcessPoolExecutor(max_workers=MAX_WORKERS)
executor

In [17]:
# Dask (Optional)
# from dask.distributed import Client
# import logging
# client = Client( silence_logs=logging.ERROR)
# executor = client.get_executor()
# client

In [18]:
# This calls `executor.map`
ev = Evaluator(function=f, executor=executor, max_workers=MAX_WORKERS)

In [19]:
# This will run in parallel
ev.evaluate_data(in10)

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

# Evaluator in the Xopt object

In [ ]:
X = Xopt(generator=RandomGenerator(vocs=vocs), evaluator=Evaluator(function=f),
         vocs=vocs)
X.strict = False

# Submit to the evaluator some new inputs
X.submit_data(vocs.random_inputs(4))

# Unevaluated inputs are collected in a dataframe
X._input_data

In [ ]:
# Internal futures dictionary
X._futures

In [ ]:
# Collect all finished futures and updata dataframe
X.process_futures()
X.data

In [ ]:
# Futures are now cleared out
X._futures

In [ ]:
# This is the internal counter
X._ix_last

In [ ]:
# This causes immediate evaluation
X.evaluate_data(vocs.random_inputs(4))

In [ ]:
# Singe generation step
X.step()
X.data

In [ ]:
# Usage with a parallel executor.
from xopt import AsynchronousXopt
X2 = AsynchronousXopt(
    generator=RandomGenerator(vocs=vocs),
    evaluator=Evaluator(function=f, executor=executor, max_workers=MAX_WORKERS),
    vocs=vocs,
)
X2.strict = False

In [ ]:
X2.step()

In [ ]:
for _ in range(20):
    X2.step()

len(X2.data)

In [ ]:
X2.data.plot.scatter("x", "y")

In [ ]:
# Asynchronous, Vectorized
X2 = AsynchronousXopt(
    generator=RandomGenerator(vocs=vocs),
    evaluator=Evaluator(function=f, executor=executor, max_workers=MAX_WORKERS),
    vocs=vocs,
)
X2.evaluator.vectorized = True
X2.strict = False

# This takes fewer steps to achieve a similar number of evaluations
for _ in range(3):
    X2.step()

len(X2.data)